In [1]:
import numpy as np
import matplotlib.pyplot as plt

from Gaussian import Gaussian

In [2]:
# Standard univariate Gaussian
test = Gaussian()
samp = test.sample(num_samp=10)
dens = test.log_p(samp)

print(test.mean)
print(test.cov)
print(test.chol)
print(samp)
print(dens)

[0.]
[[1.]]
[[1.]]
[[-1.0846114 ]
 [-0.93948803]
 [-0.65112574]
 [ 0.36127677]
 [ 0.1762524 ]
 [ 1.17868363]
 [-0.2732403 ]
 [ 0.66546057]
 [-0.48872357]
 [ 1.4993845 ]]
[-1.50712948 -1.36025741 -1.1309209  -0.98419899 -0.93447099 -1.61358608
 -0.95626867 -1.14035742 -1.0383639  -2.04301547]


In [3]:
# Multivariate Gaussian
x = Gaussian(mean = np.ones(4), store="cov")
samp = x.sample(num_samp=3)
dens = x.log_p(samp)


print("mean:")
print(x.mean)
print("cov:")
print(x.cov)
print("chol:")
print(x.chol)
print("samp:")
print(samp)
print("dens:")
print(dens)

[1. 1. 1. 1.]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 1.15000738 -0.26937802  1.83684877  0.66887176]
 [ 2.37263941  1.61736405  0.59722798  1.78665436]
 [ 0.42978571  1.22391989  0.24888376  0.6939593 ]]
[-4.89764641 -5.19891799 -4.19231462]
